## Continuation of previous notebook on PANDAS

This is the continuation of previous notebook on PANDAS

We use now two dataframes
pubs_df containing publications
affs_df containing affiliations (for all items in pubs_df, thus also Bocconi co-authors)

both loaded from csv files and with a common index UID



In [ ]:
import pandas as pd

pubs_df = pd.read_csv("wos_publications.csv")
affs_df = pd.read_csv("wos_affiliations.csv")


First step let's have a peek to the new dataset

In [ ]:
affs_df.head()

In [ ]:
affs_df.isna().sum()  #see how many nulls

In [ ]:
affs_df.isna().mean() * 100 # to show % of nulls

In [ ]:
affs_df.describe() # data composition

### Searching / selecting data data: 

Using a single colum to select data:
df[df.A > val]

to select a list of possible values use isin

df[df.A.isin(['val1', 'val2'...])

We use isin() also when the value to search is a DF or list with one only value.

In examples below we show records where the number of addresses (addr_num) is te max (99)
and where country of affiliations is ENGLAND or USA


In [ ]:

a99 =affs_df[affs_df.addr_num >=99]

affs_df[ affs_df.UID.isin( a99['UID'])].sort_values(by=['organization']).head()

In [ ]:
pubs_df[pubs_df.UID.isin( a99['UID'])]

In [ ]:
affs_df[affs_df.country.isin(['ENGLAND','USA' ])].head(10)

### Drawing an histogram

Next step is to use hist() Pandas function to draw an histogram by country

First one counts the number of countries by number of occurrences; it autosizes the steps

In [ ]:
import matplotlib.pyplot as plt
%pylab inline

grouped=affs_df.groupby('country')
# print(grouped['UID'].count())
grouped['UID'].count().hist(xrot=90, bins=10)

plt.ylabel('Countries')
plt.xlabel('N aff')
plt.title('Countries Histogram')
plt.show()



One country has a value around 6K
let's plot a reduced dataset (count >=100) 
to see which one is it

In [ ]:

g2=pd.DataFrame(grouped['UID'].count())

g2=g2[g2.UID > 99]

g2.plot(rot=90)
plt.show()


# Merge, join, append DFs

In order to combine DFs several logics are available

To apply easily on our dataset we build 2 subsets with a smaller numer of columns

pubs_df_reduced
affs_df_reduced




In [ ]:
affs_df_reduced = affs_df.sort_values(by=['UID']).loc[0:10 ,['UID', 'organization', 'country'] ]
pubs_df_reduced = pubs_df.sort_values(by=['UID']).loc[0:5 ,['UID', 'itemtitle', 'source', 'pubyear'] ]

pubs_df_reduced

### Concat()
Concatenating pandas objects together with concat():
performs concatenation operations along an axis while performing optional set logic (union or intersection) of the indexes (if any) on the other axes.

pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
          
join : {‘inner’, ‘outer’}, default ‘outer’

axis : {0/’index’, 1/’columns’}, default 0
The axis to concatenate along
( 0 --> appends vertically , 1 orizzontally)

join_axes : list of Index objects
Specific indexes to use for the other n - 1 axes instead of performing inner/outer set logic
          
sort : boolean, default None
Sort non-concatenation axis if it is not already aligned when join is ‘outer’.                   
   
ignore_index : boolean, default False
If True, do not use the index values along the concatenation axis   
   
          
let's just concat() two DFs:

the result brings all columns, with NaN value where the column is missing in one DF        

(note index column has overlapping values: to avoid add ignore_index=True)


In [ ]:
frames = [affs_df_reduced, pubs_df_reduced]

pd.concat(frames, sort = True)  


### Append()

append is a shortcut to concat() if you need to concatenate along axis=0, namely the index 


DataFrame.append(other, ignore_index=False, verify_integrity=False, sort=None)

We try the same as above, adding the ignore_index=True clause



In [ ]:
affs_df_reduced.append(pubs_df_reduced, ignore_index = True, sort=True)


## Merge()

pandas.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

how : {‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘inner’


#### Merge method    	SQL Join Name      	Description
            left    LEFT OUTER JOIN	    Uses keys from left frame only
            right	RIGHT OUTER JOIN	Uses keys from right frame only
            outer	FULL OUTER JOIN	    Uses union of keys from both frames
            inner	INNER JOIN          Uses intersection of keys from both frames


on : label or list (columns in common - These must be found in both DataFrames)

indicator : bool or str, default False: If True, adds a column to output DataFrame called “_merge” with information on the source of each row

We show now a DF merge between affs and pubs by UID


In [ ]:
pd.merge(affs_df_reduced, pubs_df_reduced, how= 'outer' , on = ['UID'])


Note: for last record we have no affiliations since aff reduced dataframe did not include it; try rerunning adding indicator = 'indicator_column' to crosscheck;

Try also using how= 'inner'

It could be also possible to set NaN to a default value with method
.fillna('value') (if more columns use a dictionary)


In [ ]:
df6 =pd.merge(affs_df_reduced, pubs_df_reduced, how= 'outer' , on = ['UID'], indicator='indicator_column').fillna({'organization': 'no org', 'country': 'no cy'})
df6

## Join

Method .join() is more convenient when mergin on index.
result = left.join(right, how='outer')

in cases where we want to merge on another column we can use .set_index() method.


In [ ]:
pubs_df_reduced.set_index('UID').join(affs_df_reduced.set_index('UID'), how='left')

## Reshaping DFs

### Stack
The stack() method “compresses” a level in the DataFrame’s columns.

the inverse operation of stack() is unstack(), which by default unstacks the last level
(needs multilevel indexed DFs)

In the following example first 10 affiliations are taken (df1) and a count by uid/country is 
built (df2)
eventually df2 has 2 keys since groupby has 2 levels; unstack creates a set of columns with country names


In [ ]:
df1 = affs_df[:10]
df2 = df1.groupby(['UID', 'country'])['organization'].count()
df2.unstack(level=-1)



In [ ]:
# we take 2 columns only and we reindex so we stack on the new index

df2=df1[['organization','UID']][:10].set_index('organization')
df2.stack()

### Pivot tables

pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')


data : DataFrame

values : column to aggregate, optional

index : column, Grouper, array, or list of the previous

If an array is passed, it must be the same length as the data. The list can contain any of the other types (except list). Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.

columns : column, Grouper, array, or list of the previous

If an array is passed, it must be the same length as the data. The list can contain any of the other types (except list). Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.

aggfunc : function, list of functions, dict, default numpy.mean

If list of functions passed, the resulting pivot table will have hierarchical columns whose top level are the function names (inferred from the function objects themselves) If dict is passed, the key is column to aggregate and value is function or list of functions

fill_value : scalar, default None
Value to replace missing values with

margins : boolean, default False
Add all row / columns (e.g. for subtotal / grand totals)

dropna : boolean, default True
Do not include columns whose entries are all NaN

margins_name : string, default ‘All’
Name of the row / column that will contain the totals when margins is True.



In [ ]:

df1 = pubs_df.loc[:15,['source', 'pubyear', 'UID']]
#df2 = df1.groupby(['source', 'pubyear']).count()
#pd.pivot_table(df2, index = ['source'], columns = ['pubyear'],  fill_value=0, dropna = False )

pd.pivot_table(df1, index = ['source'], columns = ['pubyear'],  fill_value=0, dropna = False, aggfunc='count' )


Last step:

We create a pivot table with the range of collaboration years by insitution and type of publication.

First we create a joined DF, then we slice it and eventually make a pivot

In [ ]:
import numpy as np


dfjoined= pd.merge(affs_df, pubs_df, how= 'inner' , on = ['UID'])
dfredux = dfjoined[['organization','pubyear', 'pubtype']].copy()

pd.pivot_table(dfredux, index = ['organization'], columns = ['pubtype'],
               values = ['pubyear'],fill_value=0, dropna = False,
               aggfunc={'min','max'})[0:15]



## Categorical data 

pandas can include categorical data in a DataFrame.
A categorical variable takes on a limited, and usually fixed, number of possible values



In [ ]:

df1 = pubs_df.loc[:15].copy()  #make a copy since we are going to modify
series1 = pd.DataFrame({'raw_rank': ['A', 'B', 'B', 'C', 'A', 'B', 'B', 'C', 'A', 'B', 'B', 'C',
                                     'A', 'B', 'B', 'C']})
df1['grade'] = series1['raw_rank'].astype('category')
df1['grade']



Here we rename categories (.cat.categories ) 
and we set the full range by adding missing categories (.cat.set_categories)

In [ ]:
df1["grade"].cat.categories = ["very good", "good", "bad"]
df1["grade"] = df1["grade"].cat.set_categories(["very bad", "bad", "medium",
                                                 "good", "very good"])
df1["grade"]


## Data output: csv and HDF5

Previously we imported data with read_csv() function

Inverse function is to_csv(filename)

Another option is HDFStore (Hierarchical Data Format), a dict-like object which reads and writes pandas using the high performance HDF5  format using the PyTables library (important : upgrade to PyTables >= 3.2 to avoid a bug; check with pip freeze).

DataFrame.to_hdf(path_or_buf, key, **kwargs)

pandas.read_hdf(path_or_buf, key=None, mode='r', **kwargs)


path_or_buf : str or pandas.HDFStore
File path or HDFStore object; default extension h5

key : str
Identifier for the group in the store.

mode : {‘a’, ‘w’, ‘r+’}, default ‘a’




In [ ]:
df2.to_hdf('c:\\temp\\df_test.h5', 'df2')
print('Dataframe succesfully exported!')

## Using lambda function on dataframes:

lambda functions are syntetic writing of functions that, thanks to apply method
can run across all dataframe.
A lambda function can take any number of arguments, but can only have one expression

x = lambda a, b, c : a + b + c
print(x(5, 6, 2))

df.text = df.text.apply(lambda x: x.lower())

in the example below we set lower case to organization field, then
set a new column BOC containing B if affiliation contains the word 'bocconi'
'O' else

In [ ]:
affs_df['organization']= affs_df['organization'].apply(lambda x: x.lower())
affs_df['BOC']= affs_df['organization'].apply(lambda x: 'O' if 'bocconi' not in x else 'B')
affs_df.head()